In [1]:
import torch, time
import torch.nn as nn
import torch.optim as optim

from src.utils import *
from src.override_resnet import *


class Args:
    arch = 50
    dataset = "ImageNet"
    # dataset = "CIFAR100"
    lr = 0.001
    momentum = 0.9
    batch = 16
    epochs = 10
    save_every = 1
    quan = "qat"
    only_eval = True
    verbose = True


args = Args()

In [2]:
def run_benchmark(model, img_loader):
    elapsed = 0
    model.eval()
    num_batches = 1
    # 이미지 배치들 이용하여 스크립트된 모델 실행
    for i, (images, target) in enumerate(img_loader):
        if i < num_batches:
            start = time.time()
            output = model(images)
            end = time.time()
            elapsed = elapsed + (end - start)
        else:
            break
    num_images = images.size()[0] * num_batches

    print("Elapsed time: %3.0f ms" % (elapsed / num_images * 1000))
    return elapsed

In [3]:
def fuse_model(model) -> nn.Module:
    flag = False
    for m in model.modules():
        if m.__class__.__name__ == ResNet_quan.__name__:
            if flag == True:
                raise ValueError("ResNet_quan is already fused")
            flag = True
            torch.quantization.fuse_modules(
                m,
                ["conv1", "bn1", "relu"],
                inplace=True,
            )

        if type(m) == BottleNeck_quan:
            torch.quantization.fuse_modules(
                m,
                [
                    ["conv1", "bn1", "relu1"],
                    ["conv2", "bn2", "relu2"],
                    ["conv3", "bn3"],
                ],
                inplace=True,
            )
            if m.downsample is not None:
                torch.quantization.fuse_modules(
                    m.downsample,
                    ["0", "1"],
                    inplace=True,
                )
    return model

In [4]:
# %% my code

args = Args()
# %% Load the ResNet-50 model
if args.quan == "fp32":
    # case 0 : no quantization case
    print("----------No quantization enabled")
    device = str(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
    model = layers_mapping[args.arch](
        weights=pretrained_weights_mapping[args.arch]
    ).to(device)

elif args.quan == "dynamic":
    # case 1 : Dynamic Quantization
    print("----------Dynamic Quantization enabled")
    device = "cuda"
    model = resnet50_quan(weights=pretrained_weights_mapping[args.arch]).to(device)
    quantized_model = torch.quantization.quantize_dynamic(
        model, {torch.nn.Linear}, dtype=torch.qint8
    )
    model = quantized_model

elif args.quan == "static":
    # case 2 : Static Quantization
    print("----------Static Quantization enabled")
    device = "cpu"
    model = resnet50_quan(weights=pretrained_weights_mapping[args.arch]).to(device)

elif args.quan == "qat":
    # case 3 : Quantization Aware Training
    print("----------Quantization Aware Training enabled")
    device = "cpu"
    model = resnet50_quan(weights=pretrained_weights_mapping[args.arch]).to(device)
else:
    raise ValueError("Invalid quantization method")

_folder_path = f"resnet{args.arch}_{args.dataset}" + "_" + args.quan
_file_name = (
    f"resnet{args.arch}_{args.dataset}_epoch"  # resnet18_cifar10_epoch{epoch}.pth
)

# %%Set up training and evaluation processes
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
# device = "cpu"
print(device)

----------Quantization Aware Training enabled
cpu


In [5]:
model.to(device)
model.eval()

train_loader, test_loader = GetDataset(
    dataset_name=args.dataset,
    device=device,
    root="data",
    batch_size=64,
    num_workers=8,
)

단순히 양자화 설정 방법을 변경하는 것만으로도 정확도가 67.3%를 넘을 정도로 향상이 되었습니다! 그럼에도 이 수치는 위에서 구한 기준값 71.9%에서 4퍼센트나 낮은 수치입니다. 이제 양자화 자각 학습을 시도해 봅시다.

# 5. 양자화 자각 학습(Quantization-aware training)
- 양자화 자각 학습(QAT)은 일반적으로 가장 높은 정확도를 제공하는 양자화 방법입니다. 모든 가중치화 활성값은 QAT로 인해 학습 도중에 순전파와 역전파를 도중 《가짜 양자화》됩니다. 
- 이는 float값이 int8 값으로 반올림하는 것처럼 흉내를 내지만, 모든 계산은 여전히 부동소수점 숫자로 계산을 합니다. 그래서 결국 훈련 동안의 모든 가중치 조정은 모델이 양자화될 것이라는 사실을 《자각》한 채로 이루어지게 됩니다. 
- 그래서 QAT는 양자화가 이루어지고 나면 동적 양자화나 학습 전 정적 양자화보다 대체로 더 높은 정확도를 보여줍니다.

- 실제로 QAT가 이루어지는 전체 흐름은 이전과 매우 유사합니다:

  - 이전과 같은 모델을 사용할 수 있습니다. 양자화 자각 학습을 위한 추가적인 준비는 필요 없습니다.

  - 가중치와 활성값 뒤에 어떤 종류의 가짜 양자화를 사용할 것인지 명시하는 qconfig 의 사용이 필요합니다. Observer를 명시하는 것 대신에 말이죠.

먼저 학습 함수부터 정의합니다:

In [6]:
model = fuse_model(model)

model.qconfig = torch.quantization.get_default_qconfig("x86")

model.train()

ResNet_quan(
  (conv1): ConvReLU2d(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): ReLU(inplace=True)
  )
  (bn1): Identity()
  (relu): Identity()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BottleNeck_quan(
      (conv1): ConvReLU2d(
        (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU()
      )
      (bn1): Identity()
      (conv2): ConvReLU2d(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
      )
      (bn2): Identity()
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (bn3): Identity()
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): Identity()
      )
      (relu1): Identity()
      (relu2): Identity()
      (relu3): ReLU()
      (add): FloatFunctional(
        (activation_post

In [7]:
torch.quantization.prepare_qat(model, inplace=True)
print(
    "Inverted Residual Block: After preparation for QAT, note fake-quantization modules \n",
    model.layer1,
)

Inverted Residual Block: After preparation for QAT, note fake-quantization modules 
 Sequential(
  (0): BottleNeck_quan(
    (conv1): ConvReLU2d(
      64, 64, kernel_size=(1, 1), stride=(1, 1)
      (weight_fake_quant): PerChannelMinMaxObserver(min_val=tensor([]), max_val=tensor([]))
      (activation_post_process): HistogramObserver(min_val=inf, max_val=-inf)
    )
    (bn1): Identity()
    (conv2): ConvReLU2d(
      64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
      (weight_fake_quant): PerChannelMinMaxObserver(min_val=tensor([]), max_val=tensor([]))
      (activation_post_process): HistogramObserver(min_val=inf, max_val=-inf)
    )
    (bn2): Identity()
    (conv3): Conv2d(
      64, 256, kernel_size=(1, 1), stride=(1, 1)
      (weight_fake_quant): PerChannelMinMaxObserver(min_val=tensor([]), max_val=tensor([]))
      (activation_post_process): HistogramObserver(min_val=inf, max_val=-inf)
    )
    (bn3): Identity()
    (relu): ReLU(inplace=True)
    (downsample): Sequ

/home/lee/miniconda3/envs/py312/lib/python3.12/site-packages/torch/ao/quantization/observer.py:220: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


In [8]:
print(device)

cpu


In [9]:
for epoch in range(8):
    # _, _ = SingleEpochTrain(model, train_loader, criterion, optimizer, device, verb=False)
    _, _ = SingleEpochTrain(model, test_loader, criterion, optimizer, device, verb=False)
    if epoch > 3:
        # Freeze quantizer parameters
        model.apply(torch.quantization.disable_observer)
    if epoch > 2:
        # Freeze batch norm mean and variance estimates
        model.apply(torch.nn.intrinsic.qat.freeze_bn_stats)

    tmp_model = torch.quantization.convert(model.eval(), inplace=False)
    eval_loss, eval_acc = SingleEpochEval(tmp_model, test_loader, criterion, device)
    print(f"epoch {epoch} : eval_loss {eval_loss}, eval_acc {eval_acc}")

100%|██████████| 782/782 [09:30<00:00,  1.37it/s]


epoch 0 : eval_loss 1.3583892058685918, eval_acc 80.136


100%|██████████| 782/782 [09:31<00:00,  1.37it/s]


epoch 1 : eval_loss 1.2956677321582803, eval_acc 80.27


100%|██████████| 782/782 [09:34<00:00,  1.36it/s]


...

epoch 5 : eval_loss 1.1234003631278986, eval_acc 81.064


100%|██████████| 782/782 [09:46<00:00,  1.33it/s]


epoch 6 : eval_loss 1.0897839167310148, eval_acc 81.32


 58%|█████▊    | 456/782 [43:44<31:16,  5.75s/it]  


KeyboardInterrupt: 

In [ ]:
_ = run_benchmark(model, test_loader)
print_size_of_model(model)